Import các thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

Đọc các file CAvideos.csv, USvideos.csv, GBvideos.csv và gộp thành 1 DataFrame

In [3]:
# Đường dẫn đến folder chứa các file csv, json
folder_path = 'YoutubeProject'
# Lấy tất cả các file csv trong folder
list_csv=glob.glob(folder_path+'\*.csv')
# Tạo list chứa các DataFrame
df_csv=[]
# Duyệt qua các file csv trong folder
for file in list_csv:
    # Bỏ qua các file có 2 chữ cái đầu là JP, KR, MX, RU, DE, FR, IN
    if file.split('\\')[1].split('.')[0][:2] in ['JP', 'KR', 'MX', 'RU', 'DE', 'FR', 'IN']:
        continue
    # Đọc các file csv cần sử dụng và chuyển thành DataFrame
    df=pd.read_csv(file, encoding='utf-8')
    df=df[['video_id','trending_date', 'title', 'channel_title', 'category_id',
        'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
        'thumbnail_link', 'comments_disabled', 'ratings_disabled',
        'video_error_or_removed', 'description']]
    # Thêm cột 'country' bằng cách lấy 2 chữ cái đầu trong tên file
    df['country']=file.split('\\')[1].split('.')[0][:2]
    # Append các DataFrame vào list df_csv
    df_csv.append(df)
# Nối các DataFrame trong list df_csv và gán cho biến Videos
Videos=pd.concat(df_csv, ignore_index=True)

Đọc các file CA_category_id.json, US_category_id.json, GB_category_id.json và gộp thành 1 DataFrame

In [4]:
# Lấy tất cả các file json trong folder
list_json=glob.glob(folder_path+'\*.json')
# Tạo list chứa các DataFrame
df_json=[]
# Duyệt qua các file json trong folder
for file in list_json:
    # Bỏ qua các file có 2 chữ cái đầu là JP, KR, MX, RU, DE, FR, IN
    if file.split('\\')[1].split('.')[0][:2] in ['JP', 'KR', 'MX', 'RU', 'DE', 'FR', 'IN']:
        continue
    # Đọc các file csv cần sử dụng
    df=pd.read_json(file, encoding='utf-8')
    # Lấy cột 'items' và chuyển thành DataFrame
    df=pd.DataFrame(list(df['items']))
    # Lấy cột 'snippet' trong DataFrame vừa tạo và chuyển thành DataFrame với 3 cột là 'channelID', 'title', 'assignable'
    df[['channelID', 'title', 'assignable']]=pd.DataFrame(list(df['snippet']))
    # Bỏ cột 'assignable', chỉ lấy cột 'id' và 'title'
    df=df[['id', 'title']]
    # Append các DataFrame vào list df_json
    df_json.append(df)
# Nối các DataFrame trong list df_csv và gán cho biến Videos
Category=pd.concat(df_json, ignore_index=True)

Clean data Videos

In [34]:
# Xem tổng quan về bộ dữ liệu
Videos.head()

,trending_date,title,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,country
video_id,,,,,,,,,,
n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-10-11,17158579,787425,43420,125882,CA
0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13,1014651,127794,1688,13030,CA
5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-12-11,3191434,146035,5339,8181,CA
d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-12-11,2095828,132239,1989,17518,CA
2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-09-11,33523622,1634130,21082,85067,CA


In [40]:
# Kiểm tra xem DataFrame có giá trị null không
Videos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118268 entries, n1WpP7iowLc to ooyjaVdt-jA
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   trending_date  118268 non-null  datetime64[ns]
 1   title          118268 non-null  object        
 2   channel_title  118268 non-null  object        
 3   category_id    118268 non-null  int32         
 4   publish_time   118268 non-null  datetime64[ns]
 5   views          118268 non-null  int32         
 6   likes          118268 non-null  int32         
 7   dislikes       118268 non-null  int32         
 8   comment_count  118268 non-null  int32         
 9   country        118268 non-null  object        
dtypes: datetime64[ns](2), int32(5), object(3)
memory usage: 7.7+ MB


In [7]:
# Bỏ các cột 'tags','thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description'
Videos=Videos.drop(['tags','thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description'], axis=1)

In [8]:
# Chuyển cột video_id thành index
Videos=Videos.set_index('video_id')

In [9]:
# Chuyển cột 'trending_date' về đúng định dạng date
Videos['trending_date']=Videos['trending_date'].str.split('.',expand=True)[[1,2,0]].apply(lambda x:'/'.join(x), axis=1)

In [10]:
# Chuyển cột 'publish_time' về đúng định dạng date
Videos['publish_time']=Videos['publish_time'].str.split('T',expand=True)[0].str.split('-',expand=True)[[2,1,0]].apply(lambda x:'/'.join(x), axis=1)

In [11]:
# Set kiểu dữ liệu đúng cho các cột
Videos['trending_date']=Videos['trending_date'].astype('datetime64')
Videos['title']=Videos['title'].astype(str)
Videos['channel_title']=Videos['channel_title'].astype(str)
Videos['category_id']=Videos['category_id'].astype(int)
Videos['publish_time']=Videos['publish_time'].astype('datetime64')
Videos['views']=Videos['views'].astype(int)
Videos['likes']=Videos['likes'].astype(int)
Videos['dislikes']=Videos['dislikes'].astype(int)
Videos['comment_count']=Videos['comment_count'].astype(int)
Videos['country']=Videos['country'].astype(str)

c:\Users\quann\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '13/11/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\quann\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '31/10/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\quann\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '23/10/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\quann\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '13/02/2009' in DD/MM/YYYY format. Provide format or specif

In [12]:
# Kiểm tra lại kiểu dữ liệu của DataFrame và đã xóa được các dòng trống chưa
Videos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118268 entries, n1WpP7iowLc to ooyjaVdt-jA
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   trending_date  118268 non-null  datetime64[ns]
 1   title          118268 non-null  object        
 2   channel_title  118268 non-null  object        
 3   category_id    118268 non-null  int32         
 4   publish_time   118268 non-null  datetime64[ns]
 5   views          118268 non-null  int32         
 6   likes          118268 non-null  int32         
 7   dislikes       118268 non-null  int32         
 8   comment_count  118268 non-null  int32         
 9   country        118268 non-null  object        
dtypes: datetime64[ns](2), int32(5), object(3)
memory usage: 7.7+ MB


Clean data Category

In [13]:
# Xem tổng quan về bộ dữ liệu
Category.head()

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [14]:
# Xóa bỏ các dòng bị lặp
Category.drop_duplicates(inplace=True)

In [18]:
# Kiểm tra các giá trị lặp ở cột 'title'
Category['title'].value_counts()

Comedy                   2
Film & Animation         1
Movies                   1
Trailers                 1
Shows                    1
Shorts                   1
Thriller                 1
Sci-Fi/Fantasy           1
Horror                   1
Foreign                  1
Family                   1
Drama                    1
Documentary              1
Classics                 1
Action/Adventure         1
Anime/Animation          1
Science & Technology     1
Autos & Vehicles         1
Education                1
Howto & Style            1
News & Politics          1
Entertainment            1
People & Blogs           1
Videoblogging            1
Gaming                   1
Travel & Events          1
Short Movies             1
Sports                   1
Pets & Animals           1
Music                    1
Nonprofits & Activism    1
Name: title, dtype: int64

In [33]:
# Kiểm tra id nào của thể loại comedy bị thừa và xóa bỏ
Category.drop(Category[Category['id']=='34'].index, inplace=True)

In [37]:
# Đổi tên cột 'title' thành 'category'
Category.rename(columns={'title':'category'}, inplace=True)

In [38]:
# Kiểm tra kiểu dữ liệu và các giá trị null trong các cột
Category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 78
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        31 non-null     object
 1   category  31 non-null     object
dtypes: object(2)
memory usage: 744.0+ bytes


In [43]:
# Đổi kiểu dữ liệu của cột 'id'
Category['id']=Category['id'].astype(int)